In [1]:
from pyalex import Works, Authors, Institutions, Topics, invert_abstract, Sources
import pandas as pd
from embedding_atlas.widget import EmbeddingAtlasWidget
from embedding_atlas.projection import compute_text_projection

In [3]:
sources = await Sources().get(limit=5)

In [13]:
pd.DataFrame(sources.topic_share.iloc[0])

,id,display_name,value,subfield,field,domain
0,https://openalex.org/T13881,Sociology and Norbert Elias,0.000155,"{'id': 'https://openalex.org/subfields/3312', ...","{'id': 'https://openalex.org/fields/33', 'disp...","{'id': 'https://openalex.org/domains/2', 'disp..."
1,https://openalex.org/T12830,Pectus Deformity Diagnosis and Treatment,0.000084,"{'id': 'https://openalex.org/subfields/2746', ...","{'id': 'https://openalex.org/fields/27', 'disp...","{'id': 'https://openalex.org/domains/4', 'disp..."
2,https://openalex.org/T11956,Sexual Differentiation and Disorders,0.000050,"{'id': 'https://openalex.org/subfields/1312', ...","{'id': 'https://openalex.org/fields/13', 'disp...","{'id': 'https://openalex.org/domains/1', 'disp..."
3,https://openalex.org/T12741,Maternal and Neonatal Healthcare,0.000048,"{'id': 'https://openalex.org/subfields/3600', ...","{'id': 'https://openalex.org/fields/36', 'disp...","{'id': 'https://openalex.org/domains/4', 'disp..."
4,https://openalex.org/T12750,Nigella sativa pharmacological applications,0.000048,"{'id': 'https://openalex.org/subfields/2707', ...","{'id': 'https://openalex.org/fields/27', 'disp...","{'id': 'https://openalex.org/domains/4', 'disp..."
5,https://openalex.org/T12758,Saffron Plant Research Studies,0.000047,"{'id': 'https://openalex.org/subfields/2730', ...","{'id': 'https://openalex.org/fields/27', 'disp...","{'id': 'https://openalex.org/domains/4', 'disp..."
6,https://openalex.org/T13813,Oral and Craniofacial Lesions,0.000045,"{'id': 'https://openalex.org/subfields/3616', ...","{'id': 'https://openalex.org/fields/36', 'disp...","{'id': 'https://openalex.org/domains/4', 'disp..."
7,https://openalex.org/T12869,Occupational Health and Burnout,0.000039,"{'id': 'https://openalex.org/subfields/3609', ...","{'id': 'https://openalex.org/fields/36', 'disp...","{'id': 'https://openalex.org/domains/4', 'disp..."
8,https://openalex.org/T12367,Dental Trauma and Treatments,0.000039,"{'id': 'https://openalex.org/subfields/3604', ...","{'id': 'https://openalex.org/fields/36', 'disp...","{'id': 'https://openalex.org/domains/4', 'disp..."
9,https://openalex.org/T12789,Planarian Biology and Electrostimulation,0.000039,"{'id': 'https://openalex.org/subfields/1312', ...","{'id': 'https://openalex.org/fields/13', 'disp...","{'id': 'https://openalex.org/domains/1', 'disp..."


In [ ]:
results = await Institutions().search("Swinburne University of Technology").get()
swinburne_id = results.iloc[0].id

In [135]:
works = await Works().filter_by_institution(swinburne_id).get(limit=10000)
works['abstract'] = works.abstract_inverted_index.map(invert_abstract)
works["titleabs"] = "Title: " + works.title + "\nAbstract: " + works.abstract

topics = pd.read_json("/Users/luhancheng/pyalex/topics.json")

Output()

In [136]:
works['titleabs'] = works.apply(lambda row: f"Title: {row.title}\nAbstract: {row.abstract}" if row.abstract else f"Title: {row.title}", axis=1)

In [137]:
topics = pd.json_normalize(topics.to_dict(orient="records"))
works = pd.json_normalize(works.to_dict(orient="records"))

In [139]:
topics = topics.rename(columns={"description": "text", "display_name": "name"}).loc[:, ["name", "text", "subfield.display_name", "field.display_name", "domain.display_name"]]
works = works.rename(columns={"titleabs": "text", "title": "name", "primary_topic.subfield.display_name": "subfield.display_name", "primary_topic.field.display_name": "field.display_name", "primary_topic.domain.display_name": "domain.display_name"}).loc[:, ["name", "text", "subfield.display_name", "field.display_name", "domain.display_name"]]
topics["type"] = "topic"
works["type"] = "work"
df = pd.concat([topics, works], axis=0)

In [140]:
compute_text_projection(df, "text")

In [141]:
EmbeddingAtlasWidget(df, text="name", x="projection_x", y="projection_y")

EmbeddingAtlasWidget()